In [1]:
import Modules.SQcircuit_extensions as sq_ext
import SQcircuit as sq
import numpy as np
import matplotlib.pyplot as plt
%matplotlib ipympl
import importlib
import qutip as qt
importlib.reload(sq_ext)

## Define the circuit

In [2]:
# Circuit parameters
Csh = 15
C   = 15
CJ  = 3
Lq  = 25
Lr  = 10
Δ   = 0.1
EJ  = 10.0

In [3]:
Cc = 1e-1

In [4]:
nmax_r = 3
nmax_f = 8

In [5]:
# Initialize loop(s)
loop1 = sq.Loop(0.5)  # "Value" corresponds to phiExt / phi0 threading the loop (can change later)
loop2 = sq.Loop(0.5)
loop3 = sq.Loop(0.5)

# Circuit elements
# Elements for the three qubits decoupled
C_01, C_02, C_12, L_03, L_31, L_23, JJ = [[] for _ in range(7)]
for loop in [loop1, loop2, loop3]:
    C_01.append(sq.Capacitor(C,       'fF'))
    C_02.append(sq.Capacitor(C,       'fF'))
    C_12.append(sq.Capacitor(CJ+Csh,  'fF'))
    L_03.append(sq.Inductor(Lr,       'nH'))
    L_31.append(sq.Inductor(Lq/2 - Δ, 'nH', loops=[loop]))
    L_23.append(sq.Inductor(Lq/2 + Δ, 'nH', loops=[loop]))
    JJ  .append(sq.Junction(EJ,      'GHz', loops=[loop]))
# Capacitors for coupling
C_coupling = sq.Capacitor(Cc, 'fF')

# Create the circuit
elements = {
    # qubit 1, nodes [0, 1, 2, 3]
    (0, 3): [L_03[0]],
    (0, 1): [C_01[0]],
    (0, 2): [C_02[0]],
    (3, 1): [L_31[0]],
    (1, 2): [C_12[0], JJ[0]],
    (2, 3): [L_23[0]],
    # qubit 2, nodes [0, 4, 5, 6]
    (0, 6): [L_03[1]],
    (0, 4): [C_01[1]],
    (0, 5): [C_02[1]],
    (6, 4): [L_31[1]],
    (4, 5): [C_12[1], JJ[1]],
    (5, 6): [L_23[1]],
    # qubit 3, nodes [0,7, 8, 9]
    (0, 9): [L_03[2]],
    (0, 7): [C_01[2]],
    (0, 8): [C_02[2]],
    (9, 7): [L_31[2]],
    (7, 8): [C_12[2], JJ[2]],
    (8, 9): [L_23[2]],
    # capacitive coupling
    (2, 4): [C_coupling],
    (5, 7): [C_coupling],
    (8, 1): [C_coupling],
}

qubit_qubit_qubit = sq.Circuit(elements)
qubit_qubit_qubit.description()

In [6]:
qubit_qubit_qubit.set_trunc_nums([nmax_r, nmax_r, nmax_r, nmax_f, nmax_f, nmax_f])
qubit_qubit_qubit.diag(4);

In [ ]:
qubit_qubit_qubit.efreqs-qubit_qubit_qubit.efreqs[0]

In [11]:
Cc = 2
C_R = C/2
C_C = Cc
CC = C_C
C_F = C/2 + Csh + CJ

In [12]:
C_mat_2x2_outer = np.array([    [ CC / 4 ,-CC / 4, CC / 4, CC / 4],
                                [-CC / 4 , CC / 4,-CC / 4,-CC / 4],
                                [ CC / 4 ,-CC / 4, CC / 4, CC / 4],
                                [ CC / 4 ,-CC / 4, CC / 4, CC / 4]])
C_mat_2x2_inner = np.array([    [ CC / 2 ,0      , CC / 4, CC / 4],
                                [0       , CC / 2,-CC / 4,-CC / 4],
                                [ CC / 4 ,-CC / 4, CC / 2,0      ],
                                [ CC / 4 ,-CC / 4,0      , CC / 2]])

In [9]:
C_mat_6x6 = np.zeros([6,6])

In [10]:
C_mat_6x6[:4,:4] = C_mat_2x2_outer
C_mat_6x6[2:,2:] = C_mat_2x2_outer
C_mat_6x6[2,2] = CC / 2
C_mat_6x6[3,3] = CC / 2
C_mat_6x6[2,3] = 0
C_mat_6x6[3,2] = 0
C_mat_6x6

In [7]:
print(C_mat_6x6)

In [11]:
C_eff_mat = np.diag(np.array([C_F, C_R, C_F, C_R, C_F, C_R]))

In [65]:
np.set_printoptions(linewidth=200, formatter={'float': '{:.5f}'.format})
matrix =np.linalg.inv(C_mat_6x6 + C_eff_mat)

In [66]:
matrix[0,1] ** -1

In [67]:
max_len = 0
for row in matrix:
    for num in row:
        num_str = f"{num:.5f}"  # Using 4 significant digits for this example
        if len(num_str) > max_len:
            max_len = len(num_str)

# Print the matrix with the numbers properly aligned
for row in matrix:
    for num in row:
        print(f"{num:>{max_len}.5f}", end=' ')
    print()  # New line at the end of each row

In [4]:
C_R = C/2
C_C = Cc
C_F = C/2 + Csh + CJ
C_mat = np.array([[C_R + C_C/2  , 0             , -C_C / 2      ,        0      ,-C_C / 2       , 0             ],
                  [0            , C_F + C_C/2   ,       0       , -C_C / 2      , 0             , -C_C / 2      ],
                  [-C_C / 2     ,        0      , C_R + C_C/2   , 0             ,-C_C / 2       , 0             ],
                  [            0, -C_C / 2      , 0             , C_F + C_C/2   , 0             ,-C_C / 2       ],
                  [-C_C / 2     , 0             ,-C_C / 2       , 0             , C_R + C_C/2   , 0             ],
                  [            0, -C_C / 2      , 0             ,-C_C / 2       , 0             , C_F + C_C/2   ]])

C_inv = np.linalg.inv(C_mat)
np.round(C_inv,4)

In [6]:
def hamiltonian_qubit_C_qubit_C_qubit(nmax_r, nmax_f, Cc, C = 15, CJ = 3, Csh= 15, Lq = 25, Lr = 10, Δ = 0.1):
    
    C_R = C/2
    C_C = Cc
    C_F = C/2 + Csh + CJ
    C_mat = np.array([[C_R + C_C/2  , 0             , -C_C / 2      ,        0      ,-C_C / 2       , 0             ],
                      [0            , C_F + C_C/2   ,       0       , -C_C / 2      , 0             , -C_C / 2      ],
                      [-C_C / 2     ,        0      , C_R + C_C/2   , 0             ,-C_C / 2       , 0             ],
                      [            0, -C_C / 2      , 0             , C_F + C_C/2   , 0             ,-C_C / 2       ],
                      [-C_C / 2     , 0             ,-C_C / 2       , 0             , C_R + C_C/2   , 0             ],
                      [            0, -C_C / 2      , 0             ,-C_C / 2       , 0             , C_F + C_C/2   ]])
    
    C_inv = np.linalg.inv(C_mat)
    fF = 1e-15
    
    resonator = sq_ext.sq_resonator(C_R_eff=C_inv[0,0]**-1, Lq = Lq, Lr = Lr, Δ = Δ, trunc_res =nmax_r)
    fluxonium = sq_ext.sq_fluxonium(C_F_eff=C_inv[1,1]**-1, Lq = Lq, Lr = Lr, Δ = Δ, trunc_flux=nmax_f)
    
    H_qubit = sq_ext.hamiltonian_qubit(fluxonium, resonator, Δ)
    
    I_r      = qt.identity(nmax_r)
    I_f      = qt.identity(nmax_f)
    I_qubit  = qt.identity(H_qubit .dims[0])
    
    q_r = qt.tensor(resonator.charge_op(0), I_f)
    q_f = qt.tensor(I_r, fluxonium.charge_op(0))
    
    H_0 = qt.tensor(H_qubit, I_qubit, I_qubit) + qt.tensor(I_qubit, H_qubit, I_qubit) + qt.tensor(I_qubit, I_qubit, H_qubit)
    
    C_RR = C_inv[0,2]**-1 * fF 
    C_FF = C_inv[1,3]**-1 * fF 
    H_coupling =    qt.tensor(q_r, q_r, I_qubit) / C_RR + qt.tensor(q_f, q_f, I_qubit) / C_FF + \
                    qt.tensor(I_qubit, q_r, q_r) / C_RR + qt.tensor(I_qubit, q_f, q_f) / C_FF + \
                    qt.tensor(q_r, I_qubit, q_r) / C_RR + qt.tensor(q_f, I_qubit, q_f) / C_FF
            
    return H_0 + H_coupling

In [12]:
qubit_qubit_qubit.set_trunc_nums([nmax_r, nmax_r, nmax_r, nmax_f, nmax_f, nmax_f])

In [13]:
qubit_qubit_qubit.hamiltonian().shape

In [7]:
H = hamiltonian_qubit_C_qubit_C_qubit(nmax_r, nmax_f, Cc)

In [9]:
E = sq_ext.diag(H, n_eig=4, out='GHz')[0]

In [10]:
E-E[0]

In [93]:
E-E[0]

In [109]:
qubit_qubit_qubit.efreqs-qubit_qubit_qubit.efreqs[0]

In [94]:
qubit_qubit_qubit.efreqs-qubit_qubit_qubit.efreqs[0]